https://www.kaggle.com/code/manovirat/timeseries-using-prophet-hyperparameter-tuning

In [1]:
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation
from sklearn.model_selection import ParameterGrid
from prophet.diagnostics import performance_metrics
import random
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

In [2]:
params_grid = {'seasonality_mode':('multiplicative','additive'),
               # set changepoint_prior_scale to 0.5 with steps of 0.005
               'changepoint_prior_scale':[x for x in np.arange(0.001, 0.5, 0.1)],
               'seasonality_prior_scale':[x for x in np.arange(0.01, 10, 2.5)],
              'holidays_prior_scale':[x for x in np.arange(0.01, 10, 2.5)]}

grid = ParameterGrid(params_grid)
cnt = 0
for p in grid:
    cnt = cnt+1

print('Total Possible Models',cnt)

Total Possible Models 160


In [3]:
#Define MASE Metric
def mean_absolute_scaled_error(y_true, y_pred):
    mase=0
    # Define numerator as the forecast error
    numerator = (np.abs(y_true - y_pred))

    # Define denominator as the mean absolute error of the in-sample one-step naive forecast
    y_true_ohne_1 = y_true[1:].reset_index(drop=True)
    y_true_ohne_ende = y_true[:-1].reset_index(drop=True)
    denominator = np.mean(np.abs(y_true_ohne_1 - y_true_ohne_ende))

    mase = np.mean(np.abs(numerator / denominator))

    return mase

In [4]:
# Load the data
data = pd.read_parquet('/Users/paddy/Documents/GitHub/Masterthesis_ML/data/03_30min_dataset.parquet')

# Convert the date column to datetime
data['date'] = pd.to_datetime(data['date'])


In [5]:
# Feature Engineering

# Add a column with the time
data['time'] = [x for x in range(0, len(data))]

data['hour_of_day'] = data['date'].dt.hour
data['day_of_week'] = data['date'].dt.dayofweek
data['day_of_month'] = data['date'].dt.day
data['month_of_year'] = data['date'].dt.month
data['year'] = data['date'].dt.year

# make a weekend column
data['weekend'] = 0
data.loc[data['day_of_week'] == 5, 'weekend'] = 1
data.loc[data['day_of_week'] == 6, 'weekend'] = 1

#make a monday column
data['monday'] = 0
data.loc[data['day_of_week'] == 0, 'monday'] = 1

#make a tuesday column
data['tuesday'] = 0
data.loc[data['day_of_week'] == 1, 'tuesday'] = 1

#make a wednesday column
data['wednesday'] = 0
data.loc[data['day_of_week'] == 2, 'wednesday'] = 1

#make a thursday column
data['thursday'] = 0
data.loc[data['day_of_week'] == 3, 'thursday'] = 1

#make a friday column
data['friday'] = 0
data.loc[data['day_of_week'] == 4, 'friday'] = 1

#make a saturday column
data['saturday'] = 0
data.loc[data['day_of_week'] == 5, 'saturday'] = 1

#make a sunday column
data['sunday'] = 0
data.loc[data['day_of_week'] == 6, 'sunday'] = 1

# Drop the first three rows
data = data.dropna().reset_index(drop=True)

# convert date column to datetime without timezone
data['date'] = pd.to_datetime(data['date'],format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S') 

# Rename column count to y
data = data.rename(columns={'count': 'y'})
data = data.rename(columns={'date': 'ds'})


train_data = data

In [6]:
cv_folds = TimeSeriesSplit(n_splits=5)

#convert data to only date not time
cv_data = train_data.copy()

#convert to timestamp with date format
#cv_data.ds = pd.to_datetime(cv_data.ds).dt.date

print(cv_data.ds)   

#Define cutoff dates for cross validation at end of train_data/cv_folds
cutoffs = []
# for train_index, test_index in cv_folds.split(train_data):
#     cutoffs.append(train_data.ds.iloc[test_index].max())
#     #convert to timestamp
#     cutoffs = pd.to_datetime(cutoffs)
    
# print(cutoffs)

for i, (train_index, test_index) in enumerate(cv_folds.split(cv_data)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    #get the date for the last train index
    cutoffs.append(cv_data.ds.iloc[train_index].max())
    #only date not time
    #cutoffs = pd.to_datetime(cutoffs).date

0        2022-04-01 00:00:00
1        2022-04-01 00:30:00
2        2022-04-01 01:00:00
3        2022-04-01 01:30:00
4        2022-04-01 02:00:00
                ...         
17515    2023-03-31 21:30:00
17516    2023-03-31 22:00:00
17517    2023-03-31 22:30:00
17518    2023-03-31 23:00:00
17519    2023-03-31 23:30:00
Name: ds, Length: 17520, dtype: object
Fold 0:
  Train: index=[   0    1    2 ... 2917 2918 2919]
  Test:  index=[2920 2921 2922 ... 5837 5838 5839]
Fold 1:
  Train: index=[   0    1    2 ... 5837 5838 5839]
  Test:  index=[5840 5841 5842 ... 8757 8758 8759]
Fold 2:
  Train: index=[   0    1    2 ... 8757 8758 8759]
  Test:  index=[ 8760  8761  8762 ... 11677 11678 11679]
Fold 3:
  Train: index=[    0     1     2 ... 11677 11678 11679]
  Test:  index=[11680 11681 11682 ... 14597 14598 14599]
Fold 4:
  Train: index=[    0     1     2 ... 14597 14598 14599]
  Test:  index=[14600 14601 14602 ... 17517 17518 17519]


In [7]:
MASE_list = []
best_mase = 100000000
best_params = None
for p in grid:
    train = pd.DataFrame(train_data)
    test = pd.DataFrame()
    print(p)
    random.seed(0)
    train_model =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
                        holidays_prior_scale = p['holidays_prior_scale'],
                        seasonality_prior_scale = p['seasonality_prior_scale'],
                        seasonality_mode = p['seasonality_mode'],
                        weekly_seasonality=True,
                        daily_seasonality = True,
                        yearly_seasonality = False,
                        )
    
    # add regressors
    train_model.add_regressor('hour_of_day')
    train_model.add_regressor('day_of_week')
    train_model.add_regressor('day_of_month')
    train_model.add_regressor('month_of_year')
    train_model.add_regressor('year')
    train_model.add_regressor('weekend')
    train_model.add_regressor('monday')
    train_model.add_regressor('tuesday')
    train_model.add_regressor('wednesday')
    train_model.add_regressor('thursday')
    train_model.add_regressor('friday')
    train_model.add_regressor('saturday')
    train_model.add_regressor('sunday')
    train_model.add_regressor('time')


    train_model.fit(train_data)
    
    df_cv = cross_validation(train_model, period='60 days', horizon='30 days', parallel="processes")

     # X and y to pandas dataframe
    X_train = pd.DataFrame(df_cv)

    y_true = df_cv['y'].values
    #make y_true as dataframe
    y_true = pd.DataFrame(y_true)

    y_pred = df_cv['yhat'].values
    #make y_pred as dataframe
    y_pred = pd.DataFrame(y_pred)

    MASE_list.append(mean_absolute_scaled_error(y_true, y_pred))
    


    # print('RMSE:',np.sqrt(MSE))
    # RMSE_list.append(np.sqrt(MSE))

# Find the best parameters
tuning_results = pd.DataFrame(grid)
tuning_results['mase'] = MASE_list
print(tuning_results)

{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 0.01}


19:41:01 - cmdstanpy - INFO - Chain [1] start processing
19:41:01 - cmdstanpy - INFO - Chain [1] done processing
19:41:03 - cmdstanpy - INFO - Chain [1] start processing
19:41:03 - cmdstanpy - INFO - Chain [1] start processing
19:41:03 - cmdstanpy - INFO - Chain [1] start processing
19:41:03 - cmdstanpy - INFO - Chain [1] done processing
19:41:03 - cmdstanpy - INFO - Chain [1] start processing
19:41:04 - cmdstanpy - INFO - Chain [1] start processing
19:41:04 - cmdstanpy - INFO - Chain [1] done processing
19:41:04 - cmdstanpy - INFO - Chain [1] done processing
19:41:04 - cmdstanpy - INFO - Chain [1] done processing
19:41:04 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:41:05 - cmdstanpy - INFO - Chain [1] done processing
19:41:05 - cmdstanpy - INFO - Chain [1] start processing
19:42:37 - cmdstanpy - INFO - Chain [1] done processing


{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 2.51}


19:42:38 - cmdstanpy - INFO - Chain [1] start processing
19:42:39 - cmdstanpy - INFO - Chain [1] done processing
19:42:40 - cmdstanpy - INFO - Chain [1] start processing
19:42:40 - cmdstanpy - INFO - Chain [1] start processing
19:42:40 - cmdstanpy - INFO - Chain [1] start processing
19:42:40 - cmdstanpy - INFO - Chain [1] start processing
19:42:40 - cmdstanpy - INFO - Chain [1] done processing
19:42:40 - cmdstanpy - INFO - Chain [1] done processing
19:42:40 - cmdstanpy - INFO - Chain [1] start processing
19:42:41 - cmdstanpy - INFO - Chain [1] done processing
19:42:42 - cmdstanpy - INFO - Chain [1] done processing
19:42:42 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:42:42 - cmdstanpy - INFO - Chain [1] start processing
19:42:42 - cmdstanpy - INFO - Chain [1] done processing
19:43:16 - cmdstanpy - INFO - Chain [1] done processing


{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 5.01}


...

{'changepoint_prior_scale': 0.401, 'holidays_prior_scale': 7.51, 'seasonality_mode': 'additive', 'seasonality_prior_scale': 5.01}


20:51:53 - cmdstanpy - INFO - Chain [1] start processing
20:51:57 - cmdstanpy - INFO - Chain [1] done processing
20:51:58 - cmdstanpy - INFO - Chain [1] start processing
20:51:58 - cmdstanpy - INFO - Chain [1] done processing
20:51:58 - cmdstanpy - INFO - Chain [1] start processing
20:51:58 - cmdstanpy - INFO - Chain [1] start processing
20:51:59 - cmdstanpy - INFO - Chain [1] start processing
20:51:59 - cmdstanpy - INFO - Chain [1] done processing
20:51:59 - cmdstanpy - INFO - Chain [1] start processing
20:51:59 - cmdstanpy - INFO - Chain [1] done processing
20:51:59 - cmdstanpy - INFO - Chain [1] done processing
20:52:02 - cmdstanpy - INFO - Chain [1] done processing


{'changepoint_prior_scale': 0.401, 'holidays_prior_scale': 7.51, 'seasonality_mode': 'additive', 'seasonality_prior_scale': 7.51}


20:52:03 - cmdstanpy - INFO - Chain [1] start processing
20:52:07 - cmdstanpy - INFO - Chain [1] done processing
20:52:09 - cmdstanpy - INFO - Chain [1] start processing
20:52:09 - cmdstanpy - INFO - Chain [1] start processing
20:52:09 - cmdstanpy - INFO - Chain [1] done processing
20:52:09 - cmdstanpy - INFO - Chain [1] done processing
20:52:09 - cmdstanpy - INFO - Chain [1] start processing
20:52:10 - cmdstanpy - INFO - Chain [1] start processing
20:52:10 - cmdstanpy - INFO - Chain [1] start processing
20:52:10 - cmdstanpy - INFO - Chain [1] done processing
20:52:10 - cmdstanpy - INFO - Chain [1] done processing
20:52:13 - cmdstanpy - INFO - Chain [1] done processing


     changepoint_prior_scale  holidays_prior_scale seasonality_mode  \
0                      0.001                  0.01   multiplicative   
1                      0.001                  0.01   multiplicative   
2                      0.001                  0.01   multiplicative   
3                      0.001                  0.01   multiplicative   
4                      0.001                  0.01         additive   
..                       ...                   ...              ...   
155                    0.401                  7.51   multiplicative   
156                    0.401                  7.51         additive   
157                    0.401                  7.51         additive   
158                    0.401                  7.51         additive   
159                    0.401                  7.51         additive   

     seasonality_prior_scale      mase  
0                       0.01  1.083690  
1                       2.51  0.928886  
2                       

In [8]:
best_params = grid[np.argmin(MASE_list)]
print('Best Parameters',best_params)
print('Best MASE',np.min(MASE_list))
print(MASE_list)

Best Parameters {'seasonality_prior_scale': 7.51, 'seasonality_mode': 'additive', 'holidays_prior_scale': 0.01, 'changepoint_prior_scale': 0.001}
Best MASE 0.9074312507097565
[1.0836895425312874, 0.9288857679334597, 0.9231578624162805, 0.9321591771774618, 0.9204209581930615, 0.9109197509930381, 0.910104803126744, 0.9074312507097565, 7.760365765692416, 4.031944277686117, 0.9311545830911381, 3.890248581869856, 1.0917556781735334, 7.570793625339842, 7.485561427042006, 7.486021912740025, 7.760642195087169, 6.414899177862765, 6.462781687988866, 6.278648147127525, 1.5421220383862861, 8.805958843809929, 8.826125674572477, 8.807558215099013, 7.7627737901361495, 6.958025675581833, 6.957642570975564, 6.9497825599458345, 7.81997686569928, 1.020771945198586, 1.5343662713586799, 1.136036135910256, 0.9720066857466315, 0.9751936769333881, 0.9898756879540562, 0.9218197297682166, 0.9741072262418847, 0.968491090403623, 0.9722109555438321, 0.9789130272053982, 0.9966647813248865, 0.9551084723255739, 0.989